In [39]:
## 导入现有paper的文本内容，并分章节处理文本

In [6]:
# Import the full text content and save the results separately

import csv
import os
import pickle


csv_file_path = "./Diabetes-PDFs-Outputs/all_papers.csv"
output_full_dict_path = "./data_process/ori_data/full_text_dict.pkl"
output_abstract_dict_path = "./data_process/ori_data/abstract_dict.pkl"
output_section1_dict_path = "./data_process/ori_data/introduction_dict.pkl"
output_section2_dict_path = "./data_process/ori_data/method_dict.pkl"
output_section3_dict_path = "./data_process/ori_data/result_dict.pkl"


full_text_dict = {}                 
abstract_dict = {}
section1_dict = {}
section2_dict = {}
section3_dict = {}


with open(csv_file_path, 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for idx, row in enumerate(reader):
        paper_id = f"paper_{idx + 1}"
        

        path_1 = os.path.join('./Diabetes-PDFs-Outputs/', row['PDF name'], 'pickle', row['abstract'])
        with open(path_1, 'rb') as file:
            abstract = pickle.load(file)
        

        path_2 = os.path.join('./Diabetes-PDFs-Outputs/', row['PDF name'], 'pickle', row['section 1'])
        with open(path_2, 'rb') as file:
            section1 = pickle.load(file)

        path_3 = os.path.join('./Diabetes-PDFs-Outputs/', row['PDF name'], 'pickle', row['section 2'])
        with open(path_3, 'rb') as file:
            section2 = pickle.load(file)
        
        path_4 = os.path.join('./Diabetes-PDFs-Outputs/', row['PDF name'], 'pickle', row['section 3'])
        with open(path_4, 'rb') as file:
            section3 = pickle.load(file)
        

        full_text = {
            'abstract': abstract,
            'intro': section1,
            'method': section2,
            'result': section3,
            'paper_name': row['article']
        }
        
        full_text_dict[paper_id] = full_text
        abstract_dict[paper_id] = abstract
        section1_dict[paper_id] = section1
        section2_dict[paper_id] = section2
        section3_dict[paper_id] = section3


with open(output_full_dict_path, 'wb') as file:
    pickle.dump(full_text_dict, file)
with open(output_abstract_dict_path, 'wb') as file:
    pickle.dump(abstract_dict, file)
with open(output_section1_dict_path, 'wb') as file:
    pickle.dump(section1_dict, file)
with open(output_section2_dict_path, 'wb') as file:
    pickle.dump(section2_dict, file)
with open(output_section3_dict_path, 'wb') as file:
    pickle.dump(section3_dict, file)

print("successful")


文件已成功保存。


In [41]:
# use chatgpt4 

In [7]:
from openai import OpenAI
import os

api_key = '...'
client = OpenAI(api_key=api_key)


In [8]:
# test job
completion = client.chat.completions.create(
  model="gpt-4",#o-mini-2024-07-18",
  messages=[{"role":"user","content":"hello"}]
)
print(completion.choices[0].message.content.replace('\n', ' '))


Hello! How can I assist you today?


In [9]:
import tiktoken

def split_text(text, max_tokens=5000, model="gpt-3.5-turbo"):
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    
    if len(tokens) <= max_tokens:
        return [text]
    
    splits = []
    start = 0
    while start < len(tokens):
        end = start + max_tokens
        split_tokens = tokens[start:end]
        split_texts = encoding.decode(split_tokens)
        splits.append(split_texts)
        start = end
    
    return splits


In [ ]:
# Summarize segments for each section

In [10]:

# def summarize_text(text, client, section, model="meta/llama-3.1-405b-instruct"):
#     splits = split_text(text, max_tokens=3000)
#     summaries = []
    
#     for split in splits:
#         prompt = f""" 
# You are an expert in the field of medicine with significant contributions to diabetes treatment. You are now tasked with researching and summarizing different sections of a scientific paper, highlighting the core content of the article. You need to consider the characteristics of each section of the paper to provide varying levels of content summarization.

# The main sections and themes are:

# Abstract: Briefly summarize the study's purpose, methods, results, and conclusions.
# Introduction: Provide background information, research questions, and study objectives.
# Methods: Describe the study design, experimental methods, and data collection techniques.
# Results: Present the research findings and data analysis results.
# Below is the text passage from the section {section}: {split}

# Please summarize the content according to the following requirements:

# 1. Strictly adhere to the original content of the section for summarization.
# 2. Limit the summary to within 300 words.
# 3. Use a review writing tone to appropriately refine the related text.
# 4. Do not include any words or descriptions unrelated to the original text and summary.
# """

#         completion = client.chat.completions.create(
#             model=model,
#             messages=[{"role": "user", "content": prompt}],
#             temperature=0.5,
#             top_p=1,
#             max_tokens=4500,  # Allow some buffer for response tokens
#             stream=True
#         )
#         summary = ""
#         for chunk in completion:
#             if chunk.choices[0].delta.content is not None:
#                 summary += chunk.choices[0].delta.content.replace('\n', ' ')
#         summaries.append(summary)
    
#     return summaries

# Summarize the small segments divided into sections one by one
def summarize_text(text):

    splits = split_text(text, max_tokens=3000)
    summaries = []
    
    for split in splits:
        prompt = f""" 
    You are an expert in the field of medicine and have made significant contributions to diabetes treatment. 
    Now you need to study and summarize the different parts of a scientific paper, highlighting the core content of the article. 
    You need to consider the characteristics of each part of the paper and provide different levels of content summary.

    The main chapters and topics are:
    Abstract: briefly summarize the purpose, methods, results and conclusions of the study.
    Introduction: provide background information, research questions and research objectives.
    Methods: describe the research design, experimental methods and data collection techniques.
    Results: present research findings and data analysis results.

    The following is a text paragraph: {split}
    Please summarize the content according to the following requirements:
    1. Summarize strictly in accordance with the original content of the section.
    2. Limit the summary to 300 words or less.
    3. Use a commentary writing tone to appropriately refine the relevant text.
    4. Do not include any words or descriptions that are irrelevant to the original text and summary.
    """
        completion = client.chat.completions.create(
        model="gpt-4",#o-mini-2024-07-18",
        messages=[{"role":"user","content":prompt}]
        )
        summary = completion.choices[0].message.content.replace('\n', ' ')
        summaries.append(summary)
    
    return summary


In [11]:
# Merge the segments summary list of the previous single section, and splice it by splitting spaces
def combine_summaries(summaries):
    combined_summary = " ".join(summaries)
    return combined_summary

In [ ]:
# Extract the article’s findings and prospects

In [24]:
# Extract the method and result of the text corresponding to each paperid, considering extracting from the abstract + introduction
# def extract_info(abstract, introduction, model="meta/llama-3.1-405b-instruct"):

#     prompt = f""" 
#     You are an expert in medical research, focusing on extracting key elements from academic papers. Your task is to summarize the core content related to methods and results from the given sections of a medical paper.

# Main sections and their purposes:

# 1. Abstract: Summarizes the purpose, methods, results, and conclusions of the study.
# 2. Introduction: Provides background information, research problem, and study objectives.

# The content of the abstract is: {abstract}
# The content of the introduction is: {introduction}

# Please summarize the following:

# 1. Methods: Extract and describe the research design, experimental methods, and data collection techniques used in the study.
# 2. Results: Extract and describe the key findings and data analysis results.
# Requirements:

# 1. Base the summary strictly on the original text content.
# 2. Each summary should be within 50 words.
# 3. Use a review-like tone to appropriately polish the text.
# 4. Avoid any words or descriptions unrelated to the original text and summary.
# 5. Please output in json format

# Output Example:
# "Method": "xx", 'result':"xx" 
# """

#     completion = client.chat.completions.create(
#         model=model,
#         messages=[{"role": "user", "content": prompt}],
#         temperature=0.5,
#         top_p=1,
#         max_tokens=300,  # Allow some buffer for response tokens
#         stream=True
#     )
#     info = ""
#     for chunk in completion:
#         if chunk.choices[0].delta.content is not None:
#             info += chunk.choices[0].delta.content.replace('\n', ' ')
    
#     return info

# Extract the method and result of the text corresponding to each paperid, considering extracting from the abstract + introduction
def extract_info(methods, results):

    prompt = f""" 
    You are a medical research expert who focuses on extracting key elements from academic papers. Your task is to summarize the core content related to the methods and results of a given section of a medical paper.
    Main sections and their purpose:

    Please summarize the following points:
    1. Methods: Extract and describe the research design, experimental methods, and data collection techniques used in the study.
    2. Results: Extract and describe the main findings and data analysis results.

    Requirements:
    1. The abstract is strictly based on the original text.
    2. Each abstract should be within 50 words.
    3. Use a commentary tone to polish the text appropriately.
    4. Avoid using text or descriptions that are not related to the original text and abstract.
    5. Please output in json format
    6. Generate a 50-word abstract for your summary

    Output example:
    {{ "Method": "xx", "result":"xx", "new_abstract": "xx" }}

    The original text is as follows:
    The methods are: {methods}
    The results are: {results}
"""
    
    completion = client.chat.completions.create(
    model="gpt-4",#o-mini-2024-07-18",
    messages=[{"role":"user","content":prompt}]
    )
    info = completion.choices[0].message.content.replace('\n', ' ')
    
    return info



In [ ]:
# Merge all the content to generate the final abstract.

In [14]:


# def gen_sum(text, client, section, model="meta/llama-3.1-405b-instruct"):

#     if section == 'all':
#         prompt = f""" 
#     You are an expert in the field of medicine with significant contributions to diabetes treatment. You are now tasked with researching and summarizing different sections of a scientific paper, highlighting the core content of the article. You need to consider the characteristics of each section of the paper to provide varying levels of content summarization.

#     The main sections and themes are:

#     Abstract: Briefly summarize the study's purpose, methods, results, and conclusions.
#     Introduction: Provide background information, research questions, and study objectives.
#     Methods: Describe the study design, experimental methods, and data collection techniques.
#     Results: Present the research findings and data analysis results.
#     Below is the text passage from the section {section}: {text}

#     Please summarize the content according to the following requirements:

#     1. Strictly adhere to the original content of the section for summarization.
#     2. Limit the summary to within 300 words.
#     3. Use a review writing tone to appropriately refine the related text.
#     4. Do not include any words or descriptions unrelated to the original text and summary.
#     """
        

#     else:
#         prompt = f""" 
#     You are an expert in the field of medicine with significant contributions to diabetes treatment. You are now tasked with researching and summarizing different sections of a scientific paper, highlighting the core content of the article. You need to consider the characteristics of each section of the paper to provide varying levels of content summarization.

#     The main sections and themes are:

#     Abstract: Briefly summarize the study's purpose, methods, results, and conclusions.
#     Introduction: Provide background information, research questions, and study objectives.
#     Methods: Describe the study design, experimental methods, and data collection techniques.
#     Results: Present the research findings and data analysis results.
#     Below is the text passage from the section {section}: {text}

#     Please summarize the content according to the following requirements:

#     1. Strictly adhere to the original content of the section for summarization.
#     2. Limit the summary to within 300 words.
#     3. Use a review writing tone to appropriately refine the related text.
#     4. Do not include any words or descriptions unrelated to the original text and summary.
#     """
        
#     completion = client.chat.completions.create(
#         model=model,
#         messages=[{"role": "user", "content": prompt}],
#         temperature=0.5,
#         top_p=1,
#         max_tokens=4000,  # Allow some buffer for response tokens
#         stream=True
#     )
#     summary = ""
#     for chunk in completion:
#         if chunk.choices[0].delta.content is not None:
#             summary += chunk.choices[0].delta.content.replace('\n', ' ')
    
#     return summary


# generate summary
def gen_sum(text, section):

    if section == 'all':
        prompt = f""" 
    You are an expert in the field of medicine and have made significant contributions to diabetes treatment. 
    Now you need to study and summarize the different parts of a scientific paper and highlight the core content of the article. 
    You need to consider the characteristics of each part of the paper and provide different levels of content summary.

    The main chapters and topics are:
    Abstract: briefly summarize the purpose, methods, results and conclusions of the study.
    Introduction: provide background information, research questions and research objectives.
    Methods: describe the research design, experimental methods and data collection techniques.
    Results: present research findings and data analysis results.

    The following is a text paragraph from section {section}: {text}
    Please summarize the content according to the following requirements:
    1. Strictly adhere to the original content of the section for summary.
    2. Limit the summary to 300 words or less.
    3. Use a commentary writing tone to appropriately refine the relevant text.
    4. Do not include any words or descriptions that are irrelevant to the original text and summary.

    """

    else:
        prompt = f""" 
    You are an expert in the field of medicine and have made significant contributions to diabetes treatment. 
    Now you need to study and summarize the different parts of a scientific paper and highlight the core content of the article. 
    You need to consider the characteristics of each part of the paper and provide different levels of content summary.

    The main chapters and topics are:
    Abstract: briefly summarize the purpose, methods, results and conclusions of the study.
    Introduction: provide background information, research questions and research objectives.
    Methods: describe the research design, experimental methods and data collection techniques.
    Results: present research findings and data analysis results.

    The following is a text paragraph from section {section}: {text}
    Please summarize the content according to the following requirements:
    1. Strictly adhere to the original content of the section for summary.
    2. Limit the summary to 300 words or less.
    3. Use a commentary writing tone to appropriately refine the relevant text.
    4. Do not include any words or descriptions that are irrelevant to the original text and summary.

    """
    completion = client.chat.completions.create(
    model="gpt-4",#o-mini-2024-07-18",
    messages=[{"role":"user","content":prompt}]
    )
    # print(completion)
    summary = completion.choices[0].message.content.replace('\n', ' ')
    
    return summary


In [ ]:
# For the summary of the full text, the sections are not considered, 
# The full text content is directly merged and extracted segments by segments. 
# Compare with the above segments summary and splicing content

In [16]:
# input is the full text
def generate_total_summary(client, sections):

   
    final_summary = ""
    context_summary = ""

    for i, section in enumerate(sections):
        # Combine the current paragraph with the previous paragraph's summary as context
        # prompt = f"Context summaries: {context_summary}\n\nCurrent section: {section}\n\nGenerate a summary for this section:"
        prompt = f"""
        You are a doctor who is very professional in the field of medicine. Now you are reviewing some papers in related fields. I will provide you with the original content of the current paper segment and the summary of the previous paragraph, so that you can summarize it in context and generate an English summary with less than 500 words.
        Context summary: {context_summary}\n\nCurrent paragraph: {section}\n\nGenerate a summary for this section:"""

        completion = client.chat.completions.create(
        model="gpt-4",#o-mini-2024-07-18",
        messages=[{"role":"user","content":prompt}]
        )
        # print(completion)
        section_summary = completion.choices[0].message.content.replace('\n', ' ')
        
        
        # Updated final summary and contextual summary
        final_summary += f"\nSummary for section {i+1}:\n{section_summary}\n"
        context_summary += f" {section_summary}"
        
    return final_summary


In [ ]:
# According to the full text summary, 
# get the main idea of ​​the article, 
# as well as the main advantages, disadvantages, the opinions

In [55]:
import openai

def generate_paper_summary(client, summaries, ids, themes, model='gpt-4', max_tokens=4096):

    detailed_summaries = {}

    for summary, paper_id in zip(summaries, ids):
        theme_info = themes.get(paper_id, {})
        main_theme = theme_info.get('main_theme', 'Unknown')
        sub_theme = theme_info.get('sub_theme', 'Unknown')

        prompt = f"""
        You are a professional doctor scholar who has made great achievements in the field of medicine. Now given the following paper summaries and their topics, judge whether the topics match the content of the summaries. If they match, provide a detailed analysis of the summaries, including the methods used, impact, advantages and disadvantages. If they do not match, try to do a similar summary operation yourself. Remember to use English descriptions:

        Main theme: {main_theme}
        Sub theme: {sub_theme}
        Abstract:
        {summary}
    
        Provide detailed analysis:
        """

        completion = client.chat.completions.create(
        model="gpt-4",#o-mini-2024-07-18",
        messages=[{"role":"user","content":prompt}]
        )
        detailed_summary = completion.choices[0].message.content.replace('\n', ' ')

        # Add detailed summaries to the dictionary
        detailed_summaries[paper_id] = {
            "main_theme": main_theme,
            "sub_theme": sub_theme,
            "summary": summary,
            "detailed_summary": detailed_summary
        }

    return detailed_summaries


In [ ]:
# Methods part

In [18]:
import openai

def classify_papers_by_theme(client, summaries, ids, themes, model='gpt-4', max_tokens=4096):

    theme_classification = {theme: [] for theme in themes}

    for summary, paper_id in zip(summaries, ids):
       
        prompt = f"There are the following themes:\n{themes}\n\n 
        Please help me identify which theme the following summary best matches:\n\nSummary:\n{summary}\n\nPlease only output the original text corresponding to the theme list,
        and prohibit outputting other content. The theme of this article is:"

        response = client.Completion.create(
            model=model,
            prompt=prompt,
            max_tokens=max_tokens,
            n=1,
            stop=None,
            temperature=0.5,
        )

        # Extract classification results
        classified_theme = response.choices[0].text.strip()

        # Add the paper ID to the corresponding topic
        if classified_theme in theme_classification:
            theme_classification[classified_theme].append(paper_id)
        else:
            # If the model returns a topic that is not in the list of provided topics, 
            # you can choose to ignore it or add it to a default category.
            if 'Other' not in theme_classification:
                theme_classification['Other'] = []
            theme_classification['Other'].append(paper_id)

    return theme_classification



In [ ]:
# overview of risk factors

In [19]:
import openai

def generate_risk_factor_summary(client, model, theme_to_ids, id_to_summary, max_tokens=4096):

    overall_summary = "Overview of risk factors of T2D\n"

    for theme, paper_ids in theme_to_ids.items():
        overall_summary += f"  a. {theme}\n"
        overall_summary += f"    {theme} related findings:\n"

        for paper_id in paper_ids:
            summary = id_to_summary.get(paper_id, {})
            methods = summary.get("methods", "Methods not provided.")
            results = summary.get("results", "Results not provided.")
            
            
            prompt = f"""
            Summarize the following methods and results for the main findings:
            
            Methods:
            {methods}
            
            Results:
            {results}
            
            Main Findings:
            """

            response = client.Completion.create(
                model=model,
                prompt=prompt,
                max_tokens=max_tokens,
                n=1,
                stop=None,
                temperature=0.5,
            )

            # extract summaries
            main_findings = response.choices[0].text.strip()
            overall_summary += f"    - {main_findings}\n"

    return overall_summary


## Extract content now

In [ ]:
## Input the full text and continuously generate summaries
## It needs to be segmented to avoid tokens being too long, and a summary of each segment needs to be generated

In [28]:
# Input：def generate_total_summary(client, sections): 。
# Outputs: summary 

import pickle
import json

pkl_file_path = './data_process/ori_data/full_text_dict.pkl'
with open(pkl_file_path, 'rb') as file:
    paper_dict = pickle.load(file)

paper_summary_ids = {}

# Get the full text content of each id
for paperid, value in paper_dict.items():
    full_text = paper_dict[paperid]['abstract'] + ' ' + paper_dict[paperid]['intro'] + ' ' + paper_dict[paperid]['method'] + ' ' + paper_dict[paperid]['result']
    # split text
    split_list = split_text(full_text) 
    paper_summary_ids[paperid] = split_list 


full_text_summaries = {}

for paperid, split_list in paper_summary_ids.items():
    full_text_summaries[paperid] = generate_total_summary(client, split_list)

print('Done')

Done


In [29]:
# Save the full summary of each article, and the number of words is less than 500

with open('a1-full_text_summaries.json', 'w', encoding='utf-8') as json_file:
    json.dump(full_text_summaries, json_file, ensure_ascii=False, indent=4)

In [25]:
# Operation 1: Extract the methods and results corresponding to each paperid
# Summarize the methods and results corresponding to each paperid

import pickle
pkl_file_path = './data_process/ori_data/full_text_dict.pkl'
with open(pkl_file_path, 'rb') as file:
    paper_dict = pickle.load(file)

paper_information = {}

for paperid, value in paper_dict.items():
    introduction = paper_dict[paperid]['intro']
    method = paper_dict[paperid]['method']

    gen_result = extract_info(introduction, method)
    paper_information[paperid] = gen_result

In [26]:

import json
with open('a2-paper_information.json', 'w', encoding='utf-8') as json_file:
    json.dump(paper_information, json_file, ensure_ascii=False, indent=4)

In [ ]:
# Construct rag vector database

In [41]:
!conda install faiss-gpu


^C


In [ ]:
# RAG VectorDB

import faiss
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
import pickle

# Mean Pooling - Take attention mask into account for correct averaging
def meanpooling(output, mask):
    embeddings = output[0]  # First element of model_output contains all token embeddings
    mask = mask.unsqueeze(-1).expand(embeddings.size()).float()
    return torch.sum(embeddings * mask, 1) / torch.clamp(mask.sum(1), min=1e-9)


model_name = "pubmedbert-base-embeddings" #"neuml/pubmedbert-base-embeddings"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


with open('./data_process/ori_data/full_text_dict.pkl', 'rb') as f:
    papers = pickle.load(f)


def text_to_vector(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        output = model(**inputs)
    return meanpooling(output, inputs['attention_mask']).numpy()

def split_text(text, max_length=512):
    tokens = tokenizer.tokenize(text)
    chunks = []
    for i in range(0, len(tokens), max_length):
        chunk = tokens[i:i+max_length]
        chunks.append(tokenizer.convert_tokens_to_string(chunk))
    return chunks

# Construct multiple Faiss indexes, one for each segments
index_dict = {}
text_id_dict = {}
text_content_dict = {}
original_id_map = {}  

for paper_id, sections in papers.items():
    for section_name, text in sections.items():
        if section_name == "paper_name":
            continue  
        if section_name not in index_dict:
            index_dict[section_name] = faiss.IndexFlatL2(768) 
            text_id_dict[section_name] = []
            text_content_dict[section_name] = []
        
        text_chunks = split_text(text)
        for idx, chunk in enumerate(text_chunks):
            vector = text_to_vector(chunk)
            index_dict[section_name].add(vector)
            chunk_id = f"{paper_id}_{section_name}_{idx}"
            text_id_dict[section_name].append(chunk_id)
            text_content_dict[section_name].append(chunk)
            original_id_map[chunk_id] = paper_id

# save index and text 
for section_name in index_dict:
    faiss.write_index(index_dict[section_name], f"text_vectors_{section_name}.index")
    np.save(f"text_ids_{section_name}.npy", text_id_dict[section_name])
    with open(f"text_content_{section_name}.pkl", 'wb') as f:
        pickle.dump(text_content_dict[section_name], f)
    with open(f"original_id_map_{section_name}.pkl", 'wb') as f:
        pickle.dump(original_id_map, f)


In [ ]:
# query anf rank

import faiss
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
import pickle
from FlagEmbedding import FlagReranker

# Mean Pooling - Take attention mask into account for correct averaging
def meanpooling(output, mask):
    embeddings = output[0]  # First element of model_output contains all token embeddings
    mask = mask.unsqueeze(-1).expand(embeddings.size()).float()
    return torch.sum(embeddings * mask, 1) / torch.clamp(mask.sum(1), min=1e-9)


model_name = "pubmedbert-base-embeddings" #"neuml/pubmedbert-base-embeddings"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


def text_to_vector(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        output = model(**inputs)
    return meanpooling(output, inputs['attention_mask']).numpy()


reranker = FlagReranker("bge-reranker-base", use_fp16=True)  # 'BAAI/bge-reranker-base'


index_dict = {}
text_id_dict = {}
text_content_dict = {}
original_id_map = {}
section_names = ["abstract", "intro", "method", "result"]

for section_name in section_names:
    index_dict[section_name] = faiss.read_index(f"text_vectors_{section_name}.index")
    text_id_dict[section_name] = np.load(f"text_ids_{section_name}.npy", allow_pickle=True)
    with open(f"text_content_{section_name}.pkl", 'rb') as f:
        text_content_dict[section_name] = pickle.load(f)
    with open(f"original_id_map_{section_name}.pkl", 'rb') as f:
        original_id_map.update(pickle.load(f))

# Query function
def query_vector(query, k):
    query_vector = text_to_vector(query)
    results = {}
    
    for section_name, index in index_dict.items():
        num_vectors = index.ntotal  # The number of vectors in the current index
        k_search = min(k, num_vectors)  # Make sure the search number does not exceed the number of existing vectors
        D, I = index.search(query_vector, k_search)  # Search the top k nearest neighbors
        section_results = [(original_id_map[text_id_dict[section_name][I[0][i]]], text_content_dict[section_name][I[0][i]], D[0][i]) for i in range(k_search)]
        results[section_name] = section_results
    
    return results

# Re-sorting function, in addition to outputting sorting results by chapter, 
# can also merge the total sorting results and summarize the results from multipe ways
def rerank_results(query, results):
    reranked_results = {}
    
    for section_name, section_results in results.items():
        pairs = [[query, result[1]] for result in section_results]  # Using text snippets for relevance judgment
        scores = reranker.compute_score(pairs, normalize=True)  # Calculating relevance scores
        
        if isinstance(scores, np.float64):
            scores = [scores]  # Make sure scores is a list

        scored_results = [(result[0], result[1], score) for result, score in zip(section_results, scores)]
        reranked_results[section_name] = sorted(scored_results, key=lambda x: x[2], reverse=True)  # Sort by relevance score in descending order
    
  # Merge fragments of the same original document, keeping only the first one
    final_results = {}
    for section_name, section_results in reranked_results.items():
        seen_ids = set()
        final_section_results = []
        for result in section_results:
            original_id = result[0]
            if original_id not in seen_ids:
                final_section_results.append(result)
                seen_ids.add(original_id)
        final_results[section_name] = final_section_results
    
# Calculate the average score for each paper
    paper_scores = {}
    for section_name, section_results in final_results.items():
        for result in section_results:
            paper_id = result[0]
            score = result[2]
            if paper_id not in paper_scores:
                paper_scores[paper_id] = []
            paper_scores[paper_id].append(score)
    
    average_scores = {paper_id: np.mean(scores) for paper_id, scores in paper_scores.items()}
    sorted_average_scores = sorted(average_scores.items(), key=lambda x: x[1], reverse=True)

    return final_results, sorted_average_scores


2024-07-31 00:53:53.311953: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-31 00:53:53.335761: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-31 00:53:53.813902: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# # test
# query = "A systematic review and meta-analysis was conducted to assess the relationship between the common dietary antioxidants vitamin C, vitamin E, and beta-carotene, and the incidence of cardiovascular diseases."
# initial_results = query_vector(query, k=50) # find 50 papers
# final_results, sorted_average_scores = rerank_results(query, initial_results) # Sorting: Top-k of 4 sections, compared with query
# for section_name, section_results in final_results.items():
#     print(f"Top results for section {section_name}:")
#     for result in section_results:
#         print(f"Original ID: {result[0]}, Text: {result[1]}, Score: {result[2]}")

# print("\nCombined results based on average scores:")
# for paper_id, avg_score in sorted_average_scores:
#     print(f"Paper ID: {paper_id}, Average Score: {avg_score}")

Top results for section abstract:
Original ID: paper_6, Text: a systematic review and meta - analysis was conducted to assess the relationship between the common dietary antioxidants vitamin c, vitamin e, and β - carotene and type 2 diabetes ( t2d ) and related traits. medline, embase, and the cochrane library were searched for relevant publications up until may 2023. studies were eligible if they had a cohort, case – control, or randomized controlled trial ( rct ) design and examined dietary intake, supplementation, or circulating levels of these antioxidants as exposure, and insulin resistance, β - cell function, or t2d incidence as outcomes. summary relative risks ( rr ) or mean differences ( md ) with 95 % confidence intervals ( ci ) were estimated using random - effects models. the certainty of the evidence was assessed with the grading of recommendations, assessment, development and evaluations framework. among 6190 screened records, 25 prospective observational studies and 15 rc

In [ ]:

import json

themes = {
    "Epigenetics": [
        "DNA Methylation",
        "Histone acetylation",
        "Histone Modifications",
        "RNA Interference: non-coding RNAs"
    ],
    "Genetics": [
        "Gene: TCF7L2, PPARG, KCNJ11, IRS2, HNF1A, GCK, CAPN10, TCF7L2, IRS1, IRS-2, WFS-1, HNF1B, HNF4A, TCF7L2, ABCC8, GLUT2, GCGR, FTO, MC4R, HHEX, SLC30A8, CDKN2A/B, IGF2BP2, CDKAL1, KCNQ11, NOTCH2-ADAM30...",
        "extranuclear inheritance: family history"
    ],
    "Metabolism": [
        "carbohydrate metabolites (glucose and fructose, lipid metabolites such as phospholipids, sphingomyelins, and triglycerides)",
        "amino acid metabolites (branched-chain amino acids (BCAAs), aromatic amino acids, glycine, and glutamine)"
    ],
    "Organelles Stress": [
        "Endoplasmic Reticulum (ER) Stress: Stress in the ER can lead to improper protein folding, affecting insulin production and sensitivity.",
        "Mitochondrial Stress: Dysfunctional mitochondria can impair cellular energy balance and contribute to insulin resistance.",
        "Peroxisomes Stress: Oxidative stress and metabolic disturbances affecting insulin signaling."
    ],
    "Gut Microbiome": [
        "Gut Dysbiosis: Imbalance in gut microbial communities linked to metabolic diseases, including T2D. (Bacteroides fragilis, Lactobacillus fermentum, Roseburia intestinalis, L. plantarum, L. casei, and Akkermansia muciniphila)",
        "Leaky Gut: Increased intestinal permeability, leading to systemic inflammation and insulin resistance.",
        "Gut Inflammation: Inflammatory responses in the gut influencing overall metabolic health.",
        "oxidative stress"
    ],
    "Metabolites and Metabolic Pathways": [
        "Carbohydrates: Metabolites like glucose and fructose that play a role in energy balance and insulin response.",
        "Lipids: Metabolic products such as phospholipids, sphingolipids, and triglycerides associated with insulin resistance.",
        "Amino Acids: Certain amino acids, including branched-chain amino acids (BCAAs), that may impact glucose metabolism.",
        "microbial metabolites: xanthurenate, creatine, urate, xanthine, 2-hydroxyhippurate, 3-(4-hydroxyphenyl)lactate, and 2-hydroxybutyrate"
    ],
    "Environmental Pollutants": [
        "Persistent Organic Pollutants (POPs): Chemicals like DDT, TCDD, and PCBs affecting endocrine function.",
        "Air Pollutants: Particulates such as PM2.5 and PM10 linked to increased T2D risk.",
        "Endocrine Disrupting Chemicals (EDCs): Compounds like phthalates and BPA influencing hormone regulation.",
        "Heavy Metals: Elements like cadmium (Cd), lead (Pb), arsenic (As), and mercury (Hg) that can impair metabolic health.",
        "residential noise levels",
        "neighborhood walkability",
        "greenery",
        "area-level socio-economic deprivation"
    ],
    "Diet and Lifestyle": [
        "Refined Carbohydrates: High intake of refined sugars and carbs leading to spikes in blood glucose.",
        "Saturated Fat: Diets high in saturated fats contributing to insulin resistance.",
        "Processed Foods: Consumption of highly processed foods linked to poor metabolic health.",
        "Smoking: Tobacco use increasing the risk of insulin resistance and T2D.",
        "Physical Inactivity: Lack of exercise contributing to obesity and impaired glucose metabolism.",
        "Sleep duration",
        "Antioxidants",
        "micronutrients (Vitamin B,C,D,E,... and minerals)",
        "overweight, obesity, high fat level, large waist circumference",
        "mental health: stress, depression, anxious"
    ]
}

with open('paper_themes.json', 'w', encoding='utf-8') as f:
    json.dump(themes, f, ensure_ascii=False, indent=4)


In [ ]:
import json

with open("paper_themes.json", "r") as f:
    themes = json.load(f)

# Build a topic query and get related papers
paper_theme_map = {}  # Used to record the unique topic and score corresponding to each paper ID

for theme, subthemes in themes.items():
    for subtheme in subthemes:
        results = query_vector(subtheme, k=50) 
        reranked_results, sorted_scores = rerank_results(subtheme, results)
        
        # Get the top1 paper ID and corresponding score
        top_paper_id, top_score = sorted_scores[0]
        
        # Check if the paper ID already has a corresponding subtopic
        if top_paper_id in paper_theme_map:
            # If the new score is higher, update the subtopic and score
            if top_score > paper_theme_map[top_paper_id]['score']:
                paper_theme_map[top_paper_id] = {'main_theme': theme, 'sub_theme': subtheme, 'score': top_score}
        else:
            paper_theme_map[top_paper_id] = {'main_theme': theme, 'sub_theme': subtheme, 'score': top_score}


with open("paper_theme_map.json", "w") as f:
    json.dump(paper_theme_map, f, indent=4)

print(json.dumps(paper_theme_map, indent=4))


{
    "paper_4": {
        "main_theme": "Environmental Pollutants",
        "sub_theme": "Air Pollutants: Particulates such as PM2.5 and PM10 linked to increased T2D risk.",
        "score": 0.3575504105134501
    },
    "paper_2": {
        "main_theme": "Epigenetics",
        "sub_theme": "Histone acetylation",
        "score": 0.0006556566748274668
    },
    "paper_3": {
        "main_theme": "Diet and Lifestyle",
        "sub_theme": "overweight, obesity, high fat level, large waist circumference",
        "score": 0.21734743806362997
    },
    "paper_6": {
        "main_theme": "Diet and Lifestyle",
        "sub_theme": "Smoking: Tobacco use increasing the risk of insulin resistance and T2D.",
        "score": 0.6089117101984429
    },
    "paper_5": {
        "main_theme": "Diet and Lifestyle",
        "sub_theme": "Physical Inactivity: Lack of exercise contributing to obesity and impaired glucose metabolism.",
        "score": 0.17593499173509752
    },
    "paper_1": {
     

In [ ]:
# According to the full-text summary and the existing topic, 
# obtain the key idea and other content, 
# and save it as the paper ID corresponding to the topic

In [56]:
with open("paper_theme_map.json", "r") as file:
    paper_theme_map = json.load(file)

with open("a1-full_text_summaries.json", "r") as file:
    full_text_summaries = json.load(file)


summaries = []
ids = []

for id, summary in full_text_summaries.items():
    summaries.append(summary)
    ids.append(id)

detailed_summaries = generate_paper_summary(client, summaries, ids, paper_theme_map, "gpt-4")

In [57]:
# save results

with open("a3-paper_analysis.json", "w") as f:
    json.dump(detailed_summaries, f, indent=4)

In [ ]:
# change methods

In [ ]:
# client = openai.Client(api_key="your_api_key")
# summaries = ["summary1...", "summary2...", "summary3..."]
# ids = ["paper_1", "paper_2", "paper_3"]
# themes = ["Environmental Pollutants", "Epigenetics", "Diet and Lifestyle"]

# classified_papers = classify_papers_by_theme(client, summaries, ids, themes)
# print(classified_papers)
def classify_papers_by_theme(client, summaries, ids, themes, model='gpt-4', max_tokens=4096):

In [ ]:
# Construct an overview with the following themes: 
# what someone introduced, what he used, and what he brought, and combine the information obtained from the intro, methods, and results.

In [ ]:
# summarize new version results

# client = openai.Client(api_key="your_api_key")
# theme_to_ids = {
#     "Environmental Pollutants": ["paper_1", "paper_4"],
#     "Epigenetics": ["paper_2"],
#     "Diet and Lifestyle": ["paper_3", "paper_5", "paper_6"]
# }
# id_to_summary = {
#     "paper_1": {"methods": "Methods for paper 1...", "results": "Results for paper 1..."},
#     "paper_2": {"methods": "Methods for paper 2...", "results": "Results for paper 2..."},
#     "paper_3": {"methods": "Methods for paper 3...", "results": "Results for paper 3..."},
#     "paper_4": {"methods": "Methods for paper 4...", "results": "Results for paper 4..."},
#     "paper_5": {"methods": "Methods for paper 5...", "results": "Results for paper 5..."},
#     "paper_6": {"methods": "Methods for paper 6...", "results": "Results for paper 6..."}
# }

# overall_summary = generate_risk_factor_summary(client, "gpt-4", theme_to_ids, id_to_summary)
# print(overall_summary)


def generate_risk_factor_summary(client, model, theme_to_ids, id_to_summary, max_tokens=4096):

In [ ]:
## final results

In [ ]:
## Use methods，and overview to generate conclusion，
# method:???